In [1]:
#Code to create 3 150*150 images by taking optical flow from 2 alternate frames
import os
import numpy as np
import cv2 as cv
import random
from cv2.optflow import calcOpticalFlowSF
from PIL import Image
# import IPython.display as display
from prepare_dataset import *
import glob

In [ ]:
"""
START FROM HERE
"""

In [2]:
videos_path = "/home/z3u5/Desktop/dataset_videos/"
list_of_files = glob.glob(videos_path+"*.*")
print(list_of_files)

['/home/z3u5/Desktop/dataset_videos/scene9.mp4', '/home/z3u5/Desktop/dataset_videos/scene3.mp4', '/home/z3u5/Desktop/dataset_videos/scene2.mp4', '/home/z3u5/Desktop/dataset_videos/scene10.mp4', '/home/z3u5/Desktop/dataset_videos/scene1.mp4', '/home/z3u5/Desktop/dataset_videos/scene11.mp4', '/home/z3u5/Desktop/dataset_videos/redpanda.mov', '/home/z3u5/Desktop/dataset_videos/scene7.mp4', '/home/z3u5/Desktop/dataset_videos/scene4.mp4', '/home/z3u5/Desktop/dataset_videos/scene8.mp4', '/home/z3u5/Desktop/dataset_videos/deer.mp4', '/home/z3u5/Desktop/dataset_videos/peru.mp4', '/home/z3u5/Desktop/dataset_videos/scene5.mp4', '/home/z3u5/Desktop/dataset_videos/throwingfootball.mp4', '/home/z3u5/Desktop/dataset_videos/scene6.mp4']


In [1]:
list_of_files = ['/home/z3u5/Desktop/dataset_videos/paris2.mp4']

In [4]:
frame_capture_from_videos(list_of_files) #change paths in this function to apt paths where you want to store video frames(full)

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

...

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [14]:
os.mkdir("../dataset") #place where you want to store dataset images(150*150)

In [2]:
list_of_folders = glob.glob("../training_dataset/*")#path where frames of videos are stored
print(list_of_folders)

['../training_dataset/frames_paris2']


In [3]:
print(list_of_folders)
# list_of_folders = ['../training_dataset/frames_walkingpeople'] 

['../training_dataset/frames_paris2']


In [4]:
example_number = 42168

In [7]:
"""
very important following line so that our image numberings
don't clash otherwise it will be waste
"""
# example_number = 0         uncomment this line to initialize example_number to 100000
for folder in list_of_folders:
    list_of_images = sorted(glob.glob(folder+"/*.jpg"))
#     print(len(list_of_images))
    list_of_picked_images = np.random.choice(list_of_images,int(len(list_of_images)/50),replace = False)
    for image in list_of_picked_images:
        file_index  = int(os.path.basename(os.path.splitext(image)[0]))
        print(file_index)
        if not file_index+2 > len(list_of_images)-1:
            frame0 = cv.imread(list_of_images[file_index])
            frame1 = cv.imread(list_of_images[file_index+1])
            frame2 = cv.imread(list_of_images[file_index+2])
            frames=(frame0,frame1,frame2)
#             if not is_jumpcut(frame0,frame2,threshold = 100000):
            list_of_points = create_random_crops_based_on_Prob(frame0,frame2)
            for points in list_of_points:
                save_image2(frames,points,example_number)
                example_number+=1

9340
1.520005609023475
7.315863559423798
12.843397495481804
6.837141801333034
14.92443877895944
14411
6.813758057465593
24.557536637576888
5.426838728241322
17.500167620611315
26397
4.582466204146162
3.6717467168537516
3.857172927701271
9339
12.467986666056218
31.50256546995619
1.3178871504141338
16.263264012075062
14.238687195359184
18.544424976961512
7.084018647471351
15995
4.1282287767004195
1.375860800840773
8.653883779027804
3.023513963380757
18.99496594062454
4.808836977837777
28.8868188887115
8239
23.467042895945927
6.294610132423754
7.9674209008089125
5.7228457023756025
9.612127808545216
14.985373604549627
20.47653986063629
12.653476750833898
13.00778281025017
20596
4.396436338472533
0.716582180037703
2.3831346618426315
1.0381090894020941
16.68793709275242
29071
16.496054243719847
7.837136757821415
5.9833633879676364
19.185775985113015
7.002942591980364
26408
1.5859978031756845
4.7763889124698125
4.25581087065466
5.814330728783755
6.198883964812309
5.092519154312456
4.297050995

16.742424236947848
10.32825147109403
6.228375665922856
15459
3.4377519966023673
21.089127805070085
2.956379292266817
3.3283781044327037
18453
13.408482792812563
25.031426922774095
23.49424402179948
13.837866408597916
18.132624228152164
21.049136705781525
16.28413345936544
9.76757539659013
8.901685694724234
9714
21.858708331519395
6.472156756887084
5.637822083714829
21.033418580610572
5.350464790013968
8.745351787533748
15.068019479374177
7936
16.03586029280197
12.49524493847184
17.947834587944303
14.23638904521526
4.040319231478321
23580
7.183952567129304
6.260825813105224
4.971236708823857
5.851832555562944
3.303631166034675
12.476423880742475
22077
8.165081414725242
15.682429746224278
10.169947938351653
9.01728045065591
14.280258582098465
8678
3.3536658233516703
4.764926306967612
24.105891325226533
11.16147506789051
7.461946784540266
7.682622393301143
8.51358608173581
9.460251127528709
6.759910821054888
17.451222638834253
3.747899127640115
7.362734229885077
6023
24.937944601417676
14

/home/z3u5/Desktop/Image-Interpolation-via-separable-convolution/prepare_dataset.py:52: RuntimeWarning: invalid value encountered in true_divide
  return np.linalg.norm(np.sum(flow, axis=(0, 1)) / n)


11.961543256452417
26.856116740452975
10.246258893038164
11.359763460796756
19076
10.00748176988584
28.29148566866344
4.013603354906938
1.1140319056864385
15.30931749503954
19167
11.502821590378765
6.921915031445571
10.957567539007822
13.80262890248981
10437
13.243805134041608
10.981601757987105
3.7456512812629112
16.903082704548293
3.3036683317828954
18.227258542619087
1.0819186206103908
10945
3809
28.856550250332443
9.38436177482716
25.392407746526036
19.593296342013634
1470
15.824679958724218
10.396526125905979
1.9678968910845624
8.542997446644643
7632
19.51946941310823
5.860785996301363
9.003235159585172
10.057697308216712
8.342577949861221
7.5076307963881765
10.422504162394535
12591
10.064613400441136
1.6929219269780045
28991
4.807546988345248
5.904065442811403
6.672276909844785
5.246437161964992
4.946115839481503
9.387803451657945
21058
2.2193771276138046
2630
0.3861318713072643
2.1858976739000253
2.4598324082326575
1.3555161147998638
26844
7.412281208774946
5.936976014770131
7.3

14.85050326915159
10.243753437921916
14667
14.067944738604922
11.665707595419311
26.413083139207874
7.467813939833145
17.63585287043046
12343
5.046326440135807
3.1240567317485675
9920
12.756335489723517
18.51278804414553
19.651773603110502
8.494526659364238
13.846034556094633
26.089279264185226
21.601272925405084
20664
3.331361557859348
30671
20014
11.515557644816363
19.144498863575983
4.730307274959996
29416
9.854690900561803
13.925465945630446
16.42715289265265
11.148752023901997
11.582710802562376
21.773733984833115
7.955442440757711
14.909731413517697
20.19632542249686
10870
4.295952220973186
8.492744035653535
11681
0.05088414890379486
19454
12.321054885732273
26.66153780639984
19642
12.181563498947556
16.810176329596217
1.1002944895405014
16.75577762951203
27239
10.74457861391964
9811
15.713515107668998
5.082946351176523
20.345058997937002
9.553433256054834
3710
1.3513425642724646
1.8903164643030577
1144
6.3647697015919045
5.049062450446924
4.4702804462001655
4.415604592296328
423

23.225706907630467
3.2859214891519946
3.4716530700637773
22.25573697972016
18.568371218534832
12.406839017063003
10976
2.9754580016298493
3.3778861873308834
26247
3.9307530238596753
9.947094154673556
9.61650694492822
2.867676115092023
7.627460603243704
10103
5.401232805528402
2.839761653070898
25.289980900625274
14.712329909834745
16.34756301792363
12.53195266987111
4702
16.641312774925296
2.6643965108281495
10.595003048543306
1.8256034743745992
5.899007894779253
29067
8.554205607882091
4.070179049848254
19.117934250745364
7.20729322894686
4.324421555556399
11.794507404940484
531
19.45709726960538
22.4381723542033
20.0039452199496
7083
16.843235446283614
3.7188058207731816
11.793046254188393
15.549455722123225
4.53724551387697
7362
7.447803786159587
7.132114364757025
6.720980684525965
8.399944307328523
6.665189723920327
21582
6.650135437922672
8.576418563540466
30061
9.177013028459248
6.2787677870521
2.8829157668967293
2.9770727167500852
26.195164818640915
0.2697793718264978
27.1297810

13.296885779105539
1.329737178929893
10859
11.283455120560122
17990
7.841751515180944
29.241484994183168
21.311467705410188
11.81771211438217
19162
17169
1.7554065539762138
1.6899370604172423
10009
3.397418547775851
4.9779152112508855
9.178546697264018
3612
5.287458592580688
1.5095099371064309
13917
9.562823737317553
7.994725462564598
2.0734363097628177
26737
13.652858758099166
10.743054152910792
12.866679700551098
11.003624523250751
11.025453265280893
11.066882852310203
10.668885099839184
11.827454398234682
12.703763801891693
11.601664353019276
7.958466336022771
12.843535124294162
11.631625958912648
28094
3.0376404641035846
2.118419498345996
5.354445443759234
7.564567718881918
18112
11.962582655634638
25.592255440245964
31.432313820405728
15.411416840167762
17.64494193149998
7.275798328407978
30772
0.9140332568031169
1.1770120711137013
9184
1.3307407291231057
12.969684066468686
10.662857307260348
1.541010826696439
615
5.306743673600432
2.570258240160893
2.2414501942154437
4.8734689354

1.2748923980878435
20574
11.513164844901127
5.97497816060623
1.82582812799045
7.899226410427873
1520
6.246933432909808
15.116809732935979
20.24933204076822
24849
17.929167274871826
21.186221803224893
30.96317159912175
21.49875463253962
11.52006744899311
4.5019172279316155
26.83277533423584
3995
4.893812913243239
7.998216950711286
18233
4.9338308811546625
13.162936300728708
22.913696785711142
20.717665292638838
20.24746744515479
6.738434746911655
17.035812293678724
9.852619557954133
29762
3.5289205430945803
13.620271164276403
5.024236403724812
14.567473143143403
11219
0.8892403632836724
2892
3.557033948311005
26332
7.75062699590486
7.224570187659142
7.743801415344815
5.5051583878252695
8.58432486234818
6.794307571373042
9.612839525565464
13913
14.487472842479557
18.67758954628265
18.67758954628265
10.346450415770065
18.21603844715077
2.397330649179023
20.15369005485127
13508
22.036237111309383
6.638222299188616
5.630464498118278
16.68791951218014
21.28791190881695
15.21293236769643
3.29

5.754884179980264
7.8926555278332255
1.814629606672105
13.524575388223287
28408
5.836758920982473
9.962333393297747
11.93503040870086
24912
23.864558678440634
18.621549764512125
18.72552256178299
9.458666448480619
25.052987867233124
5.229232520811653
16.414345306914303
14613
22.53390424577221
11.25006635870298
13.748963086244247
25.639487078961103
23.695935994618782
15.813053575662405
4.897842540142459
27.007325272966767
25.809334805814643
22867
1.7698465217630646
7.523625830273743
6990
13.753059633751668
12.612591619826066
9.080373052218192
11.95192866710496
6.205679030351501
14.5112395900154
9.924972193644596
7.618869363189516
12.657066141689251
6.163931388655013
22.470295704910413
19775
8.341855465208681
21.34391494324051
25.10418954086391
8.388583944674707
25.9457089590375
632
19.951437065739103
5295
5.549821879984007
2.178054948447873
14266
18.72529863293223
11.79042409533624
19.57849407826362
23.543236476755983
5.195259391787511
12.461897786741487
13.802807551044731
8.63969303920

In [8]:
print(example_number)

46532
